<a href="https://www.kaggle.com/code/putuwidyantara/question-answering" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

Source: https://huggingface.co/docs/transformers/en/tasks/question_answering

In [ ]:
!pip install -q transformers datasets evaluate huggingface_hub

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub.hf_api import HfFolder
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import create_optimizer
from transformers import TFAutoModelForQuestionAnswering
from transformers.keras_callbacks import PushToHubCallback
from transformers import pipeline

In [ ]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
HfFolder.save_token(hf_token)

In [ ]:
squad = load_dataset("squad", split="train[:5000]")

In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
callback = PushToHubCallback(
    output_dir="qa_model",
    tokenizer=tokenizer,
)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [ ]:
question_answerer = pipeline("question-answering", model="qa_model")
question_answerer(question=question, context=context)